In [ ]:
import numpy as np 
import pandas as pd 
import re
import random
import math
from tqdm.notebook import tqdm
from collections import Counter
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from transformers import RobertaTokenizer, RobertaModel
from sentence_transformers import models
from sentence_transformers import SentenceTransformer
import torch
from transformers.file_utils import is_tf_available, is_torch_available, is_torch_tpu_available
from transformers import RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer
from transformers import Trainer, TrainingArguments
import pickle
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam



In [ ]:
#################
#### DATA #######
#################


df = pd.read_csv('full_2020_bow.csv') ## Full data available on request


# Chronological sorting of articles
df['date'] = pd.to_datetime(df['article_time'])
df = df.sort_values(by='date')

import random
random.seed(1234)
article_ids = np.array(df.article_id)
articles = np.unique(article_ids)

# Splitting at the halfway point
set1 = articles[:1476]
set2 = articles[1477:]



# Getting row indices for each set of articles in original data
index1 = []
index2 = []

for row in df.itertuples():
    if row.article_id in set1:
        index1.append(row.Index)
    else:
        index2.append(row.Index)
        
df1 = df.loc[index1,:] #### DATASET 1: for train, val, test
df2 = df.loc[index2,:] #### DATASET 2: for unseen eval articles

## SPLIT DF1 IN TRAIN, TEST, VAL
np.random.seed(1)
df1 = df1.replace({np.nan:0})

train, validate, test = np.split(df1.sample(frac=1), [int(.8*len(df1)), int(.9*len(df1))])

print('train:' , collections.Counter(train['featured']))
print('validate:' , collections.Counter(validate['featured']))
print('test:' , collections.Counter(test['featured']))

###
train2 = train[train.label==0].sample(97660, random_state=1)  # DEFINE SPLIT:paper 97660, 95-5 split as calculated on validation data
train_f2 = train[train.label==1].sample(4883, random_state=1)  # DEFINE SPLIT

#train = pd.concat([train2, train_f2]) ## NEW TRAINING DATA
train2 = pd.concat([train2, train_f2]) ## NEW TRAINING DATA
train_labels = train2.featured

### processing for textual models

train_texts = list(train2.content)
test_texts = list(test.content)
val_texts = list(val.content)

test_labels = list(test.featured)
val_labels= list(validate.featured)
test_labels_temp = list(test.featured)
val_labels_temp = list(val.featured)

train_labels_num = []
for i in range(len(train_labels)):
    if train_labels[i]==True:
        train_labels_num.append(1)
    else:
        train_labels_num.append(0)


val_labels_num = []
for i in range(len(val_labels_temp)):
    if val_labels_temp[i]==True:
        val_labels_num.append(1)
    else:
        val_labels_num.append(0)


test_labels_num = []
for i in range(len(test_labels_temp)):
    if test_labels_temp[i]==True:
        test_labels_num.append(1)
    else:
        test_labels_num.append(0)


y_train = np.array(train_labels_num)
y_test = np.array(test_labels_num)
y_val = np.array(val_labels_num)


labels_df2 = np.array(labels_df2_conv)



max_words = 10000  # Maximum number of words in the vocabulary
tokenizer = Tokenizer(num_words=max_words)

all_texts = list(df.content)
tokenizer.fit_on_texts(all_texts)

sequences_train = tokenizer.texts_to_sequences(train_texts)
sequences_test = tokenizer.texts_to_sequences(test_texts)
sequences_val = tokenizer.texts_to_sequences(val_texts)

from keras.preprocessing.sequence import pad_sequences

max_sequence_length = 100  
padded_sequences_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
padded_sequences_test = pad_sequences(sequences_test, maxlen=max_sequence_length)
padded_sequences_val = pad_sequences(sequences_val, maxlen=max_sequence_length)


In [ ]:
#############################################
################## RF #######################
#############################################

## Correct_cols is used to make sure the set of variables is consistent between training and ranking
correct_cols = [
 'reply_count',
 'respect_count',
 'delta_minutes',
 'featured_posts_user',
 'rejected_count_user',
 'total_respect_user',
 'total_posts_user',
 'total_received_responses_user',
 'ratio_rejected',
 'ratio_featured',
 'ratio_reply',
 'ratio_respect',
 'wordcount',
 'intro_dist',
 'centr_dist',
 'avg_wordcount_user'
]

train_rf = train2[train2.columns.intersection(correct_cols)]
validate_rf = validate[validate.columns.intersection(correct_cols)]
test_rf = test[test.columns.intersection(correct_cols)]


# Defining grid search to be used in all RFs
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
criterion = ['gini', 'entropy']
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap, 'criterion': criterion}

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
rf_random = rf_random.fit(train_rf, train_labels)
rf1 = rf_random.best_estimator_


###### VALIDATION SET, ONLY USED WITH RF #############################


pred = rf1.predict(validate_rf)
print('f1 on validation set (RF):',f1_score(validate_labels, list(pred), pos_label=True))
print('prec on validation set (RF):',precision_score(validate_labels, list(pred), pos_label=True))
print('rec on validation set (RF):',recall_score(validate_labels, list(pred), pos_label=True))
######################################################################

# Evaluation on Test set
pred = rf1.predict(test_rf)
print('f1 on test set (RF):',f1_score(test_labels, list(pred), pos_label=True))
print('prec on test set (RF):',precision_score(test_labels, list(pred), pos_label=True))
print('rec on test set (RF):',recall_score(test_labels, list(pred), pos_label=True))

# Save RF
with open('rf', 'wb') as a:
    pickle.dump(rf1, a)

In [ ]:
## Correct cols for BoW
# Just to make sure feature set matches during ranking

correct_cols = [
 'reply_count',
 'respect_count',
 'featured',
 'delta_minutes',
 'featured_posts_user',
 'rejected_count_user',
 'total_respect_user',
 'total_posts_user',
 'total_received_responses_user',
 'ratio_rejected',
 'ratio_featured',
 'ratio_reply',
 'ratio_respect',
 'wordcount',
 'intro_dist',
 'centr_dist',
 'avg_wordcount_user',
    '000',
 '10',
 '100',
 '20',
 'aan het',
 'aan te',
 'aantal',
 'achter',
 'acties',
 'af',
 'afgelopen',
 'afstand',
 'al die',
 'alleen maar',
 'allemaal',
 'alles',
 'als de',
 'als een',
 'als er',
 'als het',
 'als ik',
 'als je',
 'als we',
 'als ze',
 'altijd',
 'ander',
 'anderen',
 'anders',
 'artikel',
 'auto',
 'bedrijf',
 'bedrijven',
 'beetje',
 'beleid',
 'bent',
 'besmet',
 'besmettingen',
 'best',
 'beste',
 'betalen',
 'beter',
 'betreft',
 'bevolking',
 'bezig',
 'bij de',
 'bijna',
 'bijvoorbeeld',
 'binnen',
 'blijft',
 'blijkbaar',
 'blijven',
 'boer',
 'br br',
 'br de',
 'br div',
 'br div div',
 'buiten',
 'burgers',
 'china',
 'cijfers',
 'corona',
 'covid',
 'daarnaast',
 'daarom',
 'dag',
 'dagen',
 'dan de',
 'dan is',
 'dan ook',
 'dat dit',
 'dat een',
 'dat er',
 'dat hij',
 'dat ik',
 'dat je',
 'dat niet',
 'dat we',
 'dat ze',
 'de maatregelen',
 'de meeste',
 'de mensen',
 'de natuur',
 'de overheid',
 'de politie',
 'de regels',
 'de regering',
 'de rest',
 'de wereld',
 'de zorg',
 'deel',
 'denk',
 'denk dat',
 'denken',
 'denkt',
 'dicht',
 'die de',
 'die het',
 'dingen',
 'dit is',
 'dit soort',
 'div br',
 'div br div',
 'div div',
 'div div br',
 'doden',
 'doe',
 'doet',
 'door de',
 'druk',
 'duidelijk',
 'duitsland',
 'echter',
 'economie',
 'een beetje',
 'een paar',
 'eerder',
 'eerst',
 'eerste',
 'eigen',
 'eigenlijk',
 'elkaar',
 'elke',
 'en als',
 'en dan',
 'en dat',
 'en die',
 'en een',
 'en het',
 'en ik',
 'en niet',
 'enige',
 'enkel',
 'enkele',
 'er een',
 'er geen',
 'er is',
 'er niet',
 'er nog',
 'er ook',
 'er zijn',
 'erg',
 'etc',
 'eten',
 'eu',
 'europa',
 'even',
 'extra',
 'feit',
 'ga',
 'gebeuren',
 'gebruiken',
 'gedaan',
 'gedrag',
 'geeft',
 'gehad',
 'geld',
 'geleden',
 'gelijk',
 'gelukkig',
 'gemaakt',
 'genoeg',
 'geval',
 'geven',
 'geweest',
 'gezegd',
 'gezien',
 'goede',
 'graag',
 'griep',
 'groep',
 'grond',
 'groot',
 'grootste',
 'grote',
 'haar',
 'had',
 'hadden',
 'halen',
 'hand',
 'hard',
 'heb ik',
 'heb je',
 'hebben we',
 'hebt',
 'heel veel',
 'helaas',
 'hele',
 'helemaal',
 'helemaal niet',
 'helpen',
 'hem',
 'het aantal',
 'het een',
 'het gaat',
 'het kabinet',
 'het land',
 'het niet',
 'het ook',
 'het probleem',
 'het rivm',
 'het virus',
 'het wel',
 'hij',
 'hoeveel',
 'hoop',
 'hoor',
 'houden',
 'huis',
 'ic',
 'idee',
 'ieder',
 'iedereen',
 'iemand',
 'iets',
 'ik ben',
 'ik denk',
 'ik denk dat',
 'ik heb',
 'ik het',
 'ik vind',
 'in een',
 'in het',
 'in nederland',
 'inderdaad',
 'is dan',
 'is dat',
 'is de',
 'is een',
 'is en',
 'is er',
 'is geen',
 'is niet',
 'is ook',
 'is voor',
 'ja',
 'jaar',
 'jaren',
 'je dat',
 'je de',
 'je een',
 'je het',
 'je niet',
 'jij',
 'juist',
 'jullie',
 'kabinet',
 'kan je',
 'kans',
 'kant',
 'keer',
 'kijk',
 'kijken',
 'kinderen',
 'kleine',
 'klopt',
 'kom',
 'komt',
 'krijgen',
 'krijgt',
 'kun',
 'kun je',
 'kunt',
 'laat',
 'laatste',
 'land',
 'landen',
 'lang',
 'laten',
 'laten we',
 'lees',
 'lekker',
 'leven',
 'liggen',
 'ligt',
 'lijkt',
 'lockdown',
 'lopen',
 'maakt',
 'maanden',
 'maar dat',
 'maar de',
 'maar een',
 'maar het',
 'maar ik',
 'maatregelen',
 'mag',
 'maken',
 'man',
 'manier',
 'me',
 'media',
 'meer dan',
 'meeste',
 'men',
 'mening',
 'mensen die',
 'met de',
 'met een',
 'met het',
 'meter',
 'mijn',
 'miljoen',
 'minder',
 'misschien',
 'moet je',
 'mogelijk',
 'mogen',
 'moment',
 'mondkapje',
 'mondkapjes',
 'mooi',
 'na',
 'naar de',
 'naar het',
 'namelijk',
 'natuur',
 'natuurlijk',
 'nederlandse',
 'nee',
 'nemen',
 'net',
 'net als',
 'niemand',
 'niet aan',
 'niet alleen',
 'niet dat',
 'niet de',
 'niet eens',
 'niet in',
 'niet meer',
 'niet op',
 'niet te',
 'niet voor',
 'niet zo',
 'niets',
 'nieuwe',
 'nieuws',
 'niks',
 'nl',
 'nodig',
 'nog een',
 'nog niet',
 'nog steeds',
 'nooit',
 'normaal',
 'nou',
 'om de',
 'om een',
 'om het',
 'om te',
 'onder',
 'onderzoek',
 'ons',
 'onze',
 'onzin',
 'ook de',
 'ook een',
 'ook niet',
 'ook nog',
 'op een',
 'op het',
 'op te',
 'open',
 'oplossing',
 'over de',
 'overheid',
 'paar',
 'partijen',
 'pas',
 'per',
 'plaats',
 'politie',
 'politiek',
 'praten',
 'precies',
 'prima',
 'probleem',
 'problemen',
 'reactie',
 'reden',
 'regels',
 'regering',
 'rest',
 'risico',
 'rivm',
 'rusland',
 'rutte',
 'scholen',
 'situatie',
 'snap',
 'snel',
 'soort',
 'staan',
 'staat',
 'steeds',
 'stemmen',
 'stikstof',
 'stoppen',
 'straks',
 'te doen',
 'te gaan',
 'te houden',
 'te maken',
 'te zijn',
 'tegen',
 'ten',
 'terug',
 'terwijl',
 'testen',
 'thuis',
 'tijd',
 'toe',
 'toen',
 'totaal',
 'trump',
 'tussen',
 'twee',
 'uit de',
 'uitstoot',
 'uw',
 'vaak',
 'vaccin',
 'valt',
 'van deze',
 'van een',
 'van het',
 'vanaf',
 'vanuit',
 'vast',
 'veel mensen',
 'vele',
 'verder',
 'vind',
 'vinden',
 'virus',
 'vlees',
 'vol',
 'volgens',
 'volgens mij',
 'voor een',
 'voor het',
 'vooral',
 'vraag',
 'waarom',
 'waarschijnlijk',
 'wanneer',
 'want',
 'waren',
 'wat een',
 'wat er',
 'week',
 'weet',
 'weg',
 'weinig',
 'weken',
 'wel een',
 'welke',
 'werd',
 'wereld',
 'werk',
 'werken',
 'werkt',
 'wet',
 'weten',
 'wie',
 'wij',
 'wil',
 'willen',
 'wilt',
 'word',
 'zaken',
 'zal',
 'ze niet',
 'zeer',
 'zeg',
 'zeggen',
 'zegt',
 'zeker',
 'zelfs',
 'zetten',
 'zie',
 'ziek',
 'ziekenhuis',
 'zien',
 'ziet',
 'zij',
 'zijn de',
 'zijn en',
 'zijn er',
 'zin',
 'zit',
 'zitten',
 'zoals',
 'zodat',
 'zonder',
 'zorg',
 'zorgen',
 'zouden',
 'zoveel',
 'zullen'
]
df1 = df1[df1.columns.intersection(correct_cols)]

In [ ]:
#############################################
################## RF_BoW ###################
#############################################

train_rfbow = train2[train2.columns.intersection(correct_cols)]
test_rfbow = test[test.columns.intersection(correct_cols)]


# Training on same grid as before
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, scoring = 'f1', verbose=2, random_state=42, n_jobs = -1)
rf_random = rf_random.fit(train_rfbow, train_labels)
rf1 = rf_random.best_estimator_

# Evaluation on Test set
pred = rf1.predict(test_rfbow)
print('f1 on test set (RF_BoW):',f1_score(test_labels_bow, list(pred), pos_label=True))
print('prec on test set (RF_BoW):',precision_score(test_labels_bow, list(pred), pos_label=True))
print('rec on test set (RF_BoW):',recall_score(test_labels_bow, list(pred), pos_label=True))


# Save rf_BoW
with open('rf_bow', 'wb') as a:
    pickle.dump(rf1, a)


In [ ]:
#############################################
################## RobBERT  #################
#############################################

# Get texts only
train_texts = list(train2['content'])
validate_texts = list(validate['content'])
test_texts = list(test['content'])


# Names
target_names = ['Not_feat', 'Feat']

# Tokenizing
tokenizer = RobertaTokenizer.from_pretrained("pdelobelle/robBERT-base")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=205)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=205)
validate_encodings = tokenizer(validate_texts, truncation=True, padding=True, max_length=205)

# Combining datasets
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# convert our tokenized data into a torch Dataset
train_dataset = Dataset(train_encodings, train_labels_num)
test_dataset = Dataset(test_encodings, test_labels_num)
validate_dataset = Dataset(validate_encodings, validate_labels_num)

# Load base model
model = RobertaForSequenceClassification.from_pretrained("pdelobelle/robBERT-base",num_labels=len(target_names))



def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds)
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    
    return {
      'accuracy': acc,
      'f1': f1,  
      'precision': precision,
      'recall': recall  
          }
# Specifics for training
training_args = TrainingArguments(
    output_dir='./results/feat_rec',          
    num_train_epochs=10,              
    per_device_train_batch_size=64,  
    per_device_eval_batch_size=64,   
    warmup_steps=250, 
    learning_rate=5e-5,
    weight_decay=0.01,               
    logging_dir='./logs/feat_rec',            
    load_best_model_at_end=True,    
    metric_for_best_model='f1',
    logging_steps=500,               
    evaluation_strategy="steps",     
)

# Trainer
trainer = Trainer(
    model=model,                         
    args=training_args,                 
    train_dataset=train_dataset,        
    eval_dataset=validate_dataset,         
    compute_metrics=compute_metrics, 
)
# Gpu
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# And training itself
trainer.train()

# Save model
model_path = "featrec"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


# On test set:
def get_prediction(text):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=205, return_tensors="pt")
    outputs = model(**inputs)
    probs = outputs[0].softmax(1)
    return probs.tolist()

test_labels2 = []
for i in range(len(test_labels_num)):
    if test_labels_num[i]==0:
        test_labels2.append('Non_feat')
    else:
        test_labels2.append('Feat')

nf = []
f = []


for i in range(len(test_texts)):
    temp = get_prediction(test_texts[i])
    temp = temp[0]
    nf.append(temp[0])
    f.append(temp[1])
    
pred = pd.DataFrame(nf)
pred['featured'] = f
pred = pred.rename(columns={0: 'Non_feat', 'featured': 'Feat'})
pred_label = []
for row in pred.itertuples():
    if row.Non_feat > row.Feat:
        pred_label.append('Non_feat')
    else:
        pred_label.append('Feat')
        
pred['Pred_label'] = pred_label
pred['Label'] = test_labels2


print('prec on test set (RobBERT):', precision_score(pred['Label'], pred['Pred_label'], pos_label='Feat'))
print('recall on test set(RobBERT):', recall_score(pred['Label'], pred['Pred_label'], pos_label='Feat'))
print('f1 on test set(RobBERT):',f1_score(pred['Label'], pred['Pred_label'], pos_label='Feat'))


In [ ]:
#############################################
################## RF_emb ###################
#############################################


# Getting CLS tokens from train and test


word_embedding_model = models.Transformer("featrec")

pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=False,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=False)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_texts_emb = list(train2.content)
test_texts_emb = list(test.content)

print('encoding')
tr_emb = []
for i in range(len(train_texts_emb)):
    temp = model.encode(train_texts_emb[i])
    tr_emb.append(temp)
print('train done')
test_emb = model.encode(test_texts_emb)

train_emb= pd.DataFrame(tr_emb)
test_emb = pd.DataFrame(test_emb)

# Combining CLS tokens with non-textual features
test_rfemb = test.drop(test.columns[0], axis=1)
train_rfemb = train2.drop(train.columns[0], axis=1)
train_rfemb = pd.concat([train_rfemb, train_emb], axis=1)
test_rfemb = pd.concat([test_rfemb, test_emb], axis=1)

# Save for replication
train2.to_csv('train_emb_full.csv')
test2.to_csv('test_emb_full.csv')

# Dropping unnecessary features
train_labels_emb = train2.featured
test_labels_emb= test2.featured
train_rfemb = train_rfemb.drop('featured', axis=1)
train_rfemb = train_rfemb.drop('content', axis=1)
test_rfemb = test_rfemb.drop('featured', axis=1)
test_rfemb = test_rfemb.drop('content', axis=1)

# Training
train2.columns = train2.columns.map(str)
test2.columns = test2.columns.map(str)

rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, scoring = 'f1', verbose=2, random_state=42, n_jobs = -1)
rf1 = rf_random.fit(train_rfemb, train_labels_emb)
pred = rf1.predict(test_rfemb)
print('f1 on test set (RF_emb):',f1_score(test_labels_emb, list(pred), pos_label=True))
print('prec on test set (RF_emb):',precision_score(test_labels_emb, list(pred), pos_label=True))
print('rec on test set (RF_emb):',recall_score(test_labels_emb, list(pred), pos_label=True))

# Save RF_EMB
with open('rf_emb', 'wb') as a:
    pickle.dump(rf1, a)

In [ ]:
##########################
###### BASELINE #########
##########################


baseline_pred = []
for row in test.itertuples():
    if row.ratio_featured>0.03:
        baseline_pred.append(True)
    else:
        baseline_pred.append(False)
print('f1 on test set(baseline):',f1_score(test_labels, baseline_pred, pos_label=True))
print('prec on test set (baseline):',precision_score(test_labels, baseline_pred, pos_label=True))
print('rec on test set (baseline):',recall_score(test_labels, baseline_pred, pos_label=True))        

In [ ]:
######################################
############## SVM #################
#####################################


svm = make_pipeline(StandardScaler(), SVC(gamma='auto', probability=True))
svm.fit(train_rf, train_labels)

test = test.drop('featured', axis=1)
pred = svm.predict(test_rf)
print('f1 on test set (SVM):',f1_score(test_labels, list(pred), pos_label=True))
print('prec on test set (SVM):',precision_score(test_labels, list(pred), pos_label=True))
print('rec on test set (SVM):',recall_score(test_labels, list(pred), pos_label=True))

In [ ]:
#############################
########## CNN ##############
############################

embedding_dim = 100  
vocab_size = len(tokenizer.word_index) + 1 

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim)) #, input_length=max_sequence_length -> deprecated
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(padded_sequences_train, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_val, y_val))

predictions = model.predict(padded_sequences_test)


preds = []
for i in range(len(predictions)):
    if predictions[i]>0.5:
        preds.append(1)
    else:
        preds.append(0)


print('f1 on test set: (CNN)',f1_score(list(y_test), preds, pos_label=1))
print('prec on test set: (CNN)',precision_score(list(y_test), preds, pos_label=1))
print('rec on test set: (CNN)',recall_score(list(y_test), preds, pos_label=1))

In [ ]:
#############################
##### BiLSTM ################
############################


max_length = 100 
input_layer = Input(shape=(max_length,))
embedding_layer = Embedding(vocab_size, 100)(input_layer)
bilstm_layer = Bidirectional(LSTM(64))(embedding_layer)
output_layer = Dense(1, activation='sigmoid')(bilstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model


model.fit(padded_sequences_train, y_train, epochs=10, batch_size=32, validation_data=(padded_sequences_val, y_val))

predictions = model.predict(padded_sequences_test)
preds = []
for i in range(len(predictions)):
    if predictions[i]>0.5:
        preds.append(1)
    else:
        preds.append(0)


print(c.Counter(preds))


print('f1 on test set: (biLSTM)',f1_score(list(y_test), preds, pos_label=1))
print('prec on test set: (biLSTM):',precision_score(list(y_test), preds, pos_label=1))
print('rec on test set: (biLSTM):',recall_score(list(y_test), preds, pos_label=1))